# Getting Started with pivmetalib

**Learning Objectives**
- Understand what PIV metadata is and why it matters
- Learn basic ontology concepts and how pivmetalib implements them
- Create your first metadata objects (people, organizations)
- Export metadata to standard formats (JSON-LD, TTL)
- Query and retrieve metadata from files

**Prerequisites**
- Python 3.9+ installed
- Basic understanding of PIV concepts
- Familiarity with basic Python (classes, dictionaries)
- 15-20 minutes of focused time

**Related Resources**
- [🚀 Quick Start Guide](QUICK_START.md) - 5-minute first example
- [📖 User Guide](USER_GUIDE.md) - Complete learning paths
- [🐛 Troubleshooting](TROUBLESHOOTING.md) - Common issues and solutions

---

## Introduction to PIV Metadata

The `pivmetalib` library provides Python classes for the **pivmeta ontology** - a standardized way to describe Particle Image Velocimetry experiments, data, and software. It helps make your PIV research **FAIR** (Findable, Accessible, Interoperable, Reusable).

### 🎯 What You'll Achieve
By the end of this tutorial, you'll be able to:
- ✅ **Create** metadata objects for people, organizations, and software
- ✅ **Export** metadata to web-friendly JSON-LD format
- ✅ **Query** existing metadata from files
- ✅ **Understand** ontology concepts and namespaces
- ✅ **Apply** these skills to your own PIV research

### 📚 Core Concepts

**Ontologies**: Standardized vocabularies for describing entities
**Classes**: Templates for creating specific types of objects
**Properties**: Attributes that describe relationships between objects
**JSON-LD**: Web-friendly format for linked data

---

## Understanding the Architecture

The `pivmetalib` builds on multiple ontologies to provide comprehensive PIV metadata support:

### 🏗️ Library Structure
- **pivmeta**: PIV-specific classes (software, hardware, experiments)
- **ontolutils.ex**: General ontology classes (PROV, DCAT, Schema.org)
- **ssnolib**: Standard name tables for consistent terminology

### 🔗 Key Ontologies Used
- **[PROV](https://www.w3.org/ns/prov/)**: Provenance (who created what, when, how)
- **[DCAT](https://www.w3.org/TR/vocab-dcat/)**: Data catalogs and datasets
- **[Schema.org](https://schema.org/)**: General web metadata
- **[M4I](http://w3id.org/nfdi4ing/metadata4ing)**: Scientific metadata for research
- **[PIVmeta](https://matthiasprobst.github.io/pivmeta/)**: PIV-specific vocabulary

Let's start by understanding how to describe a **Person** using the PROV namespace...

In [1]:
from ontolutils.ex import prov

In [2]:
creator = prov.Person(
    lastName='Okamoto',
    mbox="okamoto@tokai.t.u-tokyo.ac.jp"
)
creator

Person(id=_:N5b2adc00f4304e6a84c5ca8252d3ae01, mbox=okamoto@tokai.t.u-tokyo.ac.jp, lastName=Okamoto)

## 🎯 Quick Start: Create Your First Person

Let's create a simple person object to understand the basics:

In [3]:
print(creator.serialize("ttl"))

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix prov: <http://www.w3.org/ns/prov#> .

[] a prov:Person ;
    foaf:lastName "Okamoto" ;
    foaf:mbox "okamoto@tokai.t.u-tokyo.ac.jp" .




**What's behind the class:** Each class is inherited from the `ontolutils.Thing`, which again is a `pydantic.BaseModel` subclass. The package [`pydantic`](https://docs.pydantic.dev/latest/) allows validating the attributes of a class. This means, that passing wrong types to the person, will lead to an error:

In [4]:
import pydantic

In [5]:
try:
    prov.Person(lastName='John', mbox='email.com')
except pydantic.ValidationError as e:
    print(e)

1 validation error for Person
mbox
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='email.com', input_type=str]


**Extra fields**

Generally, it is possible to add any additional fields, like age, for example, however it will not be available in the RDF output:

In [6]:
creator = prov.Person(
    firtName='John',
    lastName='Doe',
    age=34
)
print(creator.serialize("ttl"))

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix prov: <http://www.w3.org/ns/prov#> .

[] a prov:Person ;
    foaf:lastName "Doe" .




To change this, we can update the missing URI at all times:

In [7]:
creator.add_property(
    name="age",
    property_type=int,
    namespace="https://example.org#",
    namespace_prefix="ex", default=None)

In [8]:
print(creator.serialize("ttl"))

@prefix ex: <https://example.org/#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a prov:Person ;
    foaf:lastName "Doe" ;
    ex:age 34 .




## Working with the classes

There are two importing things, you can do with the classes:
- **Dumping**: This means exporting the content to various formats (dictionary, json-ld strings, ...)
- **querying**: Finding data of a class in a JSON-LD file

### Dumping:
The most basic form of "dumping" is exporting the class instance to a dictionary:

In [9]:
creator.model_dump()

{'id': '_:N2442efff7f2242e4bb0ac255bf808ab8',
 'label': None,
 'altLabel': None,
 'broader': None,
 'comment': None,
 'about': None,
 'relation': None,
 'closeMatch': None,
 'exactMatch': None,
 'description': None,
 'isDefinedBy': None,
 'mbox': None,
 'firstName': None,
 'lastName': 'Doe',
 'orcidId': None,
 'affiliation': None,
 'firtName': 'John',
 'age': 34}

The `pivmetalib` adds also `dump_jsonld` to the classes and allows to create a JSON-LD string, which can be saved to a file, too:

In [10]:
print(creator.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcterms": "http://purl.org/dc/terms/",
        "schema": "https://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#",
        "prov": "http://www.w3.org/ns/prov#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "ex": "https://example.org/#"
    },
    "@type": "prov:Person",
    "foaf:lastName": "Doe",
    "firtName": "John",
    "ex:age": 34,
    "@id": "_:N2442efff7f2242e4bb0ac255bf808ab8"
}


**Save JSON-LD file**

In [11]:
import pivmetalib

In [12]:
# save to file:
with open('john.jsonld', 'w') as f:
    data = creator.model_dump_jsonld(context={"@import": pivmetalib.CONTEXT})
    f.write(data)

# save to file:
with open('john.ttl', 'w') as f:
    data = creator.model_dump_ttl(context={"@import": pivmetalib.CONTEXT})
    f.write(data)

Let's create another person and save both to the file. For this we will need to "merge" both JSON-LD strings. There is a helper function for this in the `utils` module: 

In [13]:
other_person = prov.Person(first_name='Lisa')

In [14]:
from pivmetalib.jsonld import merge

In [15]:
# save to file:
with open('creator.jsonld', 'w') as f:
    f.write(merge([creator.model_dump_jsonld(context={"@import": pivmetalib.CONTEXT}),
                   other_person.model_dump_jsonld(context={"@import": pivmetalib.CONTEXT})]))

### Querying

We can identify data within a JSON-LD file by calling `query` and providing the class we want to find. The method applies a SPARQL query ad returns an instance of the provided object:

In [16]:
from ontolutils import query

In [17]:
persons = query(cls=prov.Person, source='creator.jsonld', context={"@import": pivmetalib.CONTEXT})
persons

[Person(id=_:N2442efff7f2242e4bb0ac255bf808ab8, lastName=Doe, firtName=John, age=34),
 Person(id=_:N769a1e37414f491fa0e352af929cf883, firstName=Lisa)]

Likewise, you can call `from_jsonld` from the Person object:

In [18]:
persons = prov.Person.from_jsonld(source='creator.jsonld', context={"@import": pivmetalib.CONTEXT})
persons

[Person(id=_:N2442efff7f2242e4bb0ac255bf808ab8, lastName=Doe, firtName=John, age=34),
 Person(id=_:N769a1e37414f491fa0e352af929cf883, firstName=Lisa)]

## 🎉 Summary & What You've Learned

**✅ Completed Skills:**
- Creating ontology objects using pivmetalib
- Understanding PROV namespace for people and organizations
- Exporting metadata to JSON-LD format
- Validating object structure with Pydantic
- Querying metadata from JSON-LD files

**🔍 Key Concepts Mastered:**
- Ontology classes and properties
- Namespaces and IRIs
- JSON-LD serialization and context
- SPARQL querying basics

---

## 🚀 Next Steps in Your Learning Journey

### **Continue with Core PIV Tutorials:**
1. 📓 [Describe Your First PIV Recording](Describe_a_PIV_recording.ipynb) *(25 min)*
   - Document complete PIV experiments with hardware and setups
   - Create dataset descriptions with DCAT
   - Add experimental parameters and provenance

2. 🔬 [PIV Processing with OpenPIV](Describe_PIV_eval_with_openPIV.ipynb) *(30 min)*
   - Document PIV analysis workflows and processing steps
   - Capture algorithm parameters and quality metrics
   - Create reproducible research pipelines

3. 📊 [PIV Result Documentation](Describe_PIV_Result_File.ipynb) *(25 min)*
   - Document analysis results and outputs
   - Add statistical summaries and visualizations
   - Prepare data for repository submission

### **Explore Specialized Topics:**
- 🏳️ [PIV Flags & Quality Control](PIVFlags.ipynb) - Data quality management
- 🏷️ [Standard Names](StandardNameTableForPIV/Publish%20PIV%20Standard%20Name%20Table.ipynb) - Variable naming standards